In [1]:
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install tqdm
!pip install pandas
!pip install scikit-learn
!pip install nltk

You should consider upgrading via the 'C:\Users\manta\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\manta\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\manta\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\manta\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\manta\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\manta\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\manta\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import tensorflow as tf
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\manta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
MAX_LEN = 128
BATCH_SIZE = 32
NUM_EPOCHS = 3

In [4]:
data = pd.read_csv("tweets_sentiment.csv", usecols=['tweet','neg', 'neu', 'pos'])

In [5]:
data.head(5)

,tweet,neg,neu,pos
0,Bitcoin: The Cyberpunk Cryptocurrency,0.0,1.000,0.000
1,Bitcoin Isn't the Only Cryptocurrency in Town via,0.0,1.000,0.000
2,", a ""cryptocurrency"", went on a tear last week...",0.0,1.000,0.000
3,I'm going to make my OWN crypto-currency and e...,0.0,0.885,0.115
4,"Your momma's cryptocurrency is so virtual, she...",0.0,1.000,0.000


In [6]:
tweet = []
sentiment = []

In [7]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    cleaned_text = ' '.join(words)
    return cleaned_text

In [8]:
for index in data.index:
    if isinstance(data['tweet'][index], str):
        if data['neg'][index] == 0.0 and data['pos'][index] == 0.0:
            tweet.append(clean_text(data['tweet'][index]))
            sentiment.append(1)
        elif data['neg'][index] > data['pos'][index]:
            tweet.append(clean_text(data['tweet'][index]))
            sentiment.append(0)
        elif data['neg'][index] < data['pos'][index]:
            tweet.append(clean_text(data['tweet'][index]))
            sentiment.append(2)

In [9]:
data = pd.DataFrame({"text": tweet, "label": sentiment})
data = data.sample(frac = 0.1)

In [10]:
data.head(5)

,text,label
372750,told twice three times many many future didnt ...,1
665235,easily binance brazil says chad anderson one t...,2
157961,looking nano integrators itd perfect cryptocur...,2
515591,microbt maker whatsminer bitcoin mining equipm...,1
93798,crypto market hopefully recovers soon traders ...,2


In [11]:
data.shape

(82283, 2)

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [12]:
final_test_dataset = data.sample(frac =.10)

In [13]:
train = data.sample(frac = 0.70)

In [14]:
test = data.sample(frac = 0.20)

In [15]:
def tokenize_text(_data):
    input_ids = []
    attention_masks = []

    for text in tqdm(_data['text']):

        #encode text
        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LEN,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids.append(encoded_text['input_ids'])
        attention_masks.append(encoded_text['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    labels = torch.tensor(_data['label'].values)

    return input_ids, attention_masks, labels

In [16]:
train_inputs = tokenize_text(train)

train_data = TensorDataset(train_inputs[0], train_inputs[1], train_inputs[2])
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

  0%|                                                                                        | 0/57598 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\manta\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size

In [17]:
val_inputs = tokenize_text(test)

val_data = TensorDataset(val_inputs[0], val_inputs[1], val_inputs[2])
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

100%|██████████████████████████████████████████████████████████████████████████| 16457/16457 [00:10<00:00, 1576.17it/s]


In [18]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [19]:
train_encodings = tokenizer(train['text'].tolist(), truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train['label'].tolist()
)).shuffle(100).batch(16)

In [20]:
val_encodings = tokenizer(test['text'].tolist(), truncation=True, padding=True)
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    test['label'].tolist()
)).batch(16)

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [22]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

In [23]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [24]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5

3600/3600 [==============================] - 9800s 3s/step - loss: 0.4070 - accuracy: 0.8548 - val_loss: 0.2948 - val_accuracy: 0.9100
Epoch 2/5
3600/3600 [==============================] - 6897s 2s/step - loss: 0.2736 - accuracy: 0.9123 - val_loss: 0.2381 - val_accuracy: 0.9287
Epoch 3/5
3600/3600 [==============================] - 9682s 3s/step - loss: 0.2018 - accuracy: 0.9375 - val_loss: 0.2015 - val_accuracy: 0.9392
Epoch 4/5
3600/3600 [==============================] - 9646s 3s/step - loss: 0.1419 - accuracy: 0.9565 - val_loss: 0.2249 - val_accuracy: 0.9315
Epoch 5/5
3600/3600 [==============================] - 15163s 4s/step - loss: 0.0974 - accuracy: 0.9701 - val_loss: 0.1792 - val_accuracy: 0.9507


In [25]:
preds = []

In [26]:
new_encodings = tokenizer(final_test_dataset['text'].tolist(), truncation=True, padding=True)

new_dataset = tf.data.Dataset.from_tensor_slices(dict(new_encodings)).batch(16)

predictions = model.predict(new_dataset)

predicted_classes = tf.argmax(predictions.logits, axis=1)

for predicted_class in predicted_classes:
    preds.append(predicted_class)

515/515 [==============================] - 335s 646ms/step


In [27]:
from sklearn.metrics import classification_report
print(classification_report(final_test_dataset['label'].tolist(), preds))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90      1408
           1       0.96      0.93      0.94      2136
           2       0.94      0.97      0.96      4684

    accuracy                           0.95      8228
   macro avg       0.94      0.92      0.93      8228
weighted avg       0.94      0.95      0.94      8228



In [28]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
print(accuracy_score(final_test_dataset['label'].tolist(), preds) * 100)
print(precision_score(final_test_dataset['label'].tolist(), preds, average='macro') * 100)
print(f1_score(final_test_dataset['label'].tolist(), preds, average='macro') * 100)
print(recall_score(final_test_dataset['label'].tolist(), preds, average='macro') * 100)

94.50656295576081
94.37007739008291
93.36754984849553
92.46463826573529


In [29]:
model.save_pretrained('TFDistillBERTFineTuned')

In [30]:
!zip -r model.zip TFDistillBERTFineTuned

'zip' is not recognized as an internal or external command,
operable program or batch file.


In [31]:
model_ = TFDistilBertForSequenceClassification.from_pretrained('TFDistillBERTFineTuned')

Some layers from the model checkpoint at TFDistillBERTFineTuned were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at TFDistillBERTFineTuned and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
preds = []

In [33]:
new_encodings = tokenizer(final_test_dataset['text'].tolist(), truncation=True, padding=True)

new_dataset = tf.data.Dataset.from_tensor_slices(dict(new_encodings)).batch(16)

predictions = model_.predict(new_dataset)

predicted_classes = tf.argmax(predictions.logits, axis=1)

for predicted_class in predicted_classes:
    preds.append(predicted_class)

515/515 [==============================] - 340s 655ms/step


In [34]:
from sklearn.metrics import classification_report
print(classification_report(final_test_dataset['label'].tolist(), preds))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90      1408
           1       0.96      0.93      0.94      2136
           2       0.94      0.97      0.96      4684

    accuracy                           0.95      8228
   macro avg       0.94      0.92      0.93      8228
weighted avg       0.94      0.95      0.94      8228



In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(final_test_dataset['label'].tolist(), preds)

array([[1221,   34,  153],
       [  26, 1992,  118],
       [  66,   55, 4563]], dtype=int64)